In [1]:
import yfinance as yf
import pathlib
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from backtesting_v2_9 import *

%load_ext autoreload
%autoreload 2

In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
RAW_PRICES_DIR = DATA_DIR / 'prices' / 'raw'
COMPILED_PRICES_DIR = DATA_DIR / 'prices' / 'compiled'

with open(COMPILED_PRICES_DIR / 'compiled.pkl', 'rb') as f:
    prices_df = pickle.load(f)

In [3]:
SIM_DATE_FROM = "2021-12-01"
SIM_DATE_TO = "2024-05-31"
CDI = 11.59
INITIAL_CAPITAL = 100000

In [4]:
prices_df

,RAIZ4.SA,TOTS3.SA,CPLE6.SA,ITUB4.SA,SMFT3.SA,GGBR4.SA,ELET3.SA,PETZ3.SA,PETR4.SA,WEGE3.SA,...,SOMA3.SA,RENT3.SA,GMAT3.SA,VALE3.SA,SMTO3.SA,JBSS3.SA,SBSP3.SA,ARZZ3.SA,VIVA3.SA,RADL3.SA
0,4.923124,28.930948,5.128630,19.539875,16.659586,16.114634,31.568790,18.324335,11.870057,31.075983,...,12.677909,47.320843,5.661659,57.687241,31.882263,30.082554,31.784008,64.599976,23.172520,20.379776
1,5.102473,29.377386,5.262932,20.278564,17.071175,17.032127,33.255669,17.956856,12.775752,31.295971,...,12.413173,50.428009,5.721360,60.365013,32.644024,31.082943,34.348442,67.965347,22.695717,20.772228
2,5.398399,29.600601,5.212569,20.208214,17.874754,17.149120,34.113567,18.999704,12.955945,31.353357,...,13.217185,51.498138,5.860662,59.034340,32.383907,29.577929,34.928715,69.732635,23.401382,21.323544
3,5.228017,29.717064,5.153812,20.445650,18.315744,17.549372,34.393112,18.791134,13.014510,31.573351,...,13.364262,51.413662,6.069616,62.237808,33.238556,29.870083,35.387318,71.175529,24.135654,21.295507
4,5.308724,30.454651,5.237750,20.190624,18.119749,17.826466,33.872585,19.585688,13.226236,33.055897,...,13.638802,50.587589,6.258670,62.697800,33.350037,30.463230,35.555786,70.237274,23.840038,21.753378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,2.830000,29.680000,9.400000,32.141708,22.330000,18.450001,36.139999,3.960000,36.810001,39.029999,...,6.030000,45.700001,7.620000,65.050003,27.340000,29.480000,75.360001,50.950001,22.150000,26.660000
618,2.830000,29.059999,9.300000,31.831884,21.860001,18.719999,36.200001,3.830000,36.610001,38.340000,...,6.070000,45.400002,7.510000,65.080002,26.910000,29.360001,75.610001,50.959999,22.209999,26.059999
619,2.900000,28.820000,9.300000,31.721947,21.600000,18.510000,36.200001,3.800000,37.009998,38.349998,...,6.030000,44.939999,7.560000,65.300003,27.049999,28.959999,75.070000,50.970001,22.490000,25.920000
620,2.850000,28.500000,9.310000,31.552042,22.000000,18.469999,35.860001,3.800000,37.799999,38.230000,...,6.060000,44.560001,7.500000,63.889999,27.500000,28.950001,74.629997,50.990002,22.480000,25.830000


In [5]:
benchmark_ticker = 'BOVA11.SA'
data = yf.download(benchmark_ticker, start=SIM_DATE_FROM, end=SIM_DATE_TO)
bova = data.dropna()

# Save to csv in prices folder
bova.to_csv(f'{benchmark_ticker}.csv')

[*********************100%%**********************]  1 of 1 completed


In [6]:
bova

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-01,99.260002,100.180000,96.910004,97.089996,97.089996,14177070
2021-12-02,98.449997,100.430000,98.300003,100.430000,100.430000,11747920
2021-12-03,100.860001,102.839996,100.150002,101.320000,101.320000,11461680
2021-12-06,101.980003,103.570000,101.680000,103.000000,103.000000,9256574
2021-12-07,104.500000,104.500000,103.120003,103.629997,103.629997,7162804
...,...,...,...,...,...,...
2024-05-23,122.000000,122.050003,120.800003,121.099998,121.099998,5528040
2024-05-24,121.269997,121.639999,120.629997,120.639999,120.639999,3789370
2024-05-27,120.870003,120.940002,120.449997,120.870003,120.870003,2204985


In [7]:
tickers = ['BOVA11.SA']

prices = pd.DataFrame()
for ticker in tickers:
    prices[ticker] = pd.read_csv('{}.csv'.format(ticker))['Adj Close'].iloc[252:]

prices = prices.dropna()
prices

,BOVA11.SA
252,105.629997
253,106.739998
254,105.589996
255,103.570000
256,103.900002
...,...
617,121.099998
618,120.639999
619,120.870003
620,120.180000


In [18]:
from backtesting_v2_9 import *

tickers = ['BOVA11.SA', 'BOVA11.SA']

def test():
    # Se os dados estão sincronizados
    reference = prices[tickers[0]].index[-1]
    sync = True
    for ticker in tickers:
        if prices[ticker].index[-1] != reference:
            sync = False

    if sync and len(prices[tickers[0]]) == 253:
            
        for ticker in tickers:
            print(f'{ticker}: {1000/prices[ticker].Close.iloc[-1]}')
            BUY(symbol=ticker, quantity=1000/prices[ticker].Close.iloc[-1])
    

print(simple_test(tickers, SIM_DATE_FROM, SIM_DATE_TO, test, risk_free_rate=CDI))

Chega
Open         9.926000e+01
High         1.001800e+02
Low          9.691000e+01
Close        9.709000e+01
Adj Close    9.709000e+01
Volume       1.417707e+07
Name: 2021-12-01 00:00:00, dtype: float64
BOVA11.SA


IndexError: invalid index to scalar variable.